In [2]:
from tools.pos_encoder import *
from tools.time_encoder import *

In [3]:
import logging
import pandas as pd
import numpy as np
import time
import datetime
import pickle
import random
import os
LOG_FORMAT = "%(asctime)s - %(levelname)s - %(message)s"
# logging.basicConfig(level=logging.INFO, filename='log_Model', format=LOG_FORMAT)
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)

from IPython.display import display, HTML
def show_df(df):
    display(HTML(df.to_html()))
    
# VAL_POINT = 65623

# 读取数据

In [4]:
pickle_path = './1_build_dataset/df_train_info.pickle'
with open(pickle_path, 'rb') as f:
    df_train_info = pickle.load(f)

In [5]:

logging.info('start read data')
pickle_path = './0_data_sort_out/train_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_mp = pickle.load(f)
key_list_train, mp_action_train, mp_distance_train, mp_order_train, mp_distance_train_detail = train_dataset_mp['key_list'], train_dataset_mp['mp_action'], train_dataset_mp['mp_distance'], train_dataset_mp['mp_order'], train_dataset_mp['mp_distance_detail']

pickle_path = './0_data_sort_out/testA_dataset_mp.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_mp = pickle.load(f)
key_list_testA, mp_action_testA, mp_distance_testA, mp_order_testA, mp_distance_testA_detail = testA_dataset_mp['key_list'], testA_dataset_mp['mp_action'], testA_dataset_mp['mp_distance'], testA_dataset_mp['mp_order'], testA_dataset_mp['mp_distance_detail']

pickle_path = './0_data_sort_out/train_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    train_dataset_df = pickle.load(f)
df_actions_train, df_couriers_train, df_distances_train, df_orders_train = train_dataset_df['df_actions'], train_dataset_df['df_couriers'], train_dataset_df['df_distances'], train_dataset_df['df_orders']
    
pickle_path = './0_data_sort_out/testA_dataset_df.pickle'
with open(pickle_path, 'rb') as f:
    testA_dataset_df = pickle.load(f)
df_actions_testA, df_couriers_testA, df_distances_testA, df_orders_testA = testA_dataset_df['df_actions'], testA_dataset_df['df_couriers'], testA_dataset_df['df_distances'], testA_dataset_df['df_orders']

pickle_path = './2_generate_train_test_courier_feature/mp_couriers_features_train.pickle'
with open(pickle_path, 'rb') as f:
    mp_couriers_features_train = pickle.load(f)

logging.info('finish read data')

2021-01-19 16:28:36,884 - INFO - start read data
2021-01-19 16:29:00,743 - INFO - finish read data


In [6]:
# 初始化
def build_dict(mp, date, courier, wave_idx):
    if date not in mp:
        mp[date] = {}
    if courier not in mp[date]:
        mp[date][courier] = {}
    if wave_idx not in mp[date][courier]:
         mp[date][courier][wave_idx] = {}
    return mp

#数据分块
def pick_part_features(s, e):
    mp_action_part, mp_distance_part, mp_order_part, mp_couriers_part = {}, {}, {}, {}
    for i in range(s, e):
        date, courier, wave_idx = key_list_train[i]
        mp_action_part = build_dict(mp_action_part, date, courier, wave_idx)
        mp_distance_part = build_dict(mp_distance_part, date, courier, wave_idx)
        mp_order_part = build_dict(mp_order_part, date, courier, wave_idx)        
        mp_action_part[date][courier][wave_idx] = mp_action_train[date][courier][wave_idx]
        mp_distance_part[date][courier][wave_idx] = mp_distance_train_detail[date][courier][wave_idx]
        mp_order_part[date][courier][wave_idx] = mp_order_train[date][courier][wave_idx]
        if courier not in mp_couriers_part:
            mp_couriers_part[courier] = {}
        mp_couriers_part[courier][date] = mp_couriers_features_train[courier][date]
        #courier 没有wave[courier][date]
    key_list = key_list_train[s : e]
    
    mp_info_dict = {
        'action' : mp_action_part,
        'distance' : mp_distance_part,
        'order' : mp_order_part,
        'couriers' : mp_couriers_part,
    }
    data_info_dict = {
        'know_lens_list' : df_train_info['know_lens_list'].values[s:e],
        'full_lens' : df_train_info['lens'].values[s:e],
        'impossible_idxs_list' : df_train_info['impossible_idxs_list'].values[s:e]
    }    
    
    return key_list, mp_info_dict, data_info_dict

part = 32
len_key_list = len(key_list_train)
s_list, e_list = [], []

for i in range(0, len_key_list, int(len_key_list / part)):
    start, end = i, i + int(len_key_list / part)
    end = min(end, len_key_list)
    logging.info(('%d, %d') %(start,end))
    key_list, mp_info_dict, data_info_dict = pick_part_features(start, end)
    pickle_path = './3_part_train_mp/keylist_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(key_list, f)
        
    pickle_path = './3_part_train_mp/mp_info_dict_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(mp_info_dict, f)
            
    pickle_path = './3_part_train_mp/data_info_dict_%d_%d.pickle' % (start, end)
    with open(pickle_path, 'wb') as f:
        pickle.dump(data_info_dict, f)

2021-01-19 16:29:00,766 - INFO - 0, 76
2021-01-19 16:29:00,944 - INFO - 76, 152
2021-01-19 16:29:01,070 - INFO - 152, 228
2021-01-19 16:29:01,294 - INFO - 228, 304
2021-01-19 16:29:01,430 - INFO - 304, 380
2021-01-19 16:29:01,566 - INFO - 380, 456
2021-01-19 16:29:01,707 - INFO - 456, 532
2021-01-19 16:29:01,874 - INFO - 532, 608
2021-01-19 16:29:01,978 - INFO - 608, 684
2021-01-19 16:29:02,101 - INFO - 684, 760
2021-01-19 16:29:02,228 - INFO - 760, 836
2021-01-19 16:29:02,800 - INFO - 836, 912
2021-01-19 16:29:02,966 - INFO - 912, 988
2021-01-19 16:29:03,093 - INFO - 988, 1064
2021-01-19 16:29:03,216 - INFO - 1064, 1140
2021-01-19 16:29:03,356 - INFO - 1140, 1216
2021-01-19 16:29:03,484 - INFO - 1216, 1292
2021-01-19 16:29:03,619 - INFO - 1292, 1368
2021-01-19 16:29:03,739 - INFO - 1368, 1444
2021-01-19 16:29:03,917 - INFO - 1444, 1520
2021-01-19 16:29:04,028 - INFO - 1520, 1596
2021-01-19 16:29:04,649 - INFO - 1596, 1672
2021-01-19 16:29:04,811 - INFO - 1672, 1748
2021-01-19 16:29:04

In [7]:
key_list

[[Timestamp('2020-02-01 00:00:00'), 125119169, 3],
 [Timestamp('2020-02-01 00:00:00'), 125119169, 4],
 [Timestamp('2020-02-01 00:00:00'), 125155433, 0],
 [Timestamp('2020-02-01 00:00:00'), 125155433, 1],
 [Timestamp('2020-02-01 00:00:00'), 125157297, 0],
 [Timestamp('2020-02-01 00:00:00'), 125157297, 1],
 [Timestamp('2020-02-01 00:00:00'), 125157297, 2],
 [Timestamp('2020-02-01 00:00:00'), 125157297, 3],
 [Timestamp('2020-02-01 00:00:00'), 125387458, 0],
 [Timestamp('2020-02-01 00:00:00'), 125552498, 0],
 [Timestamp('2020-02-01 00:00:00'), 125552498, 1],
 [Timestamp('2020-02-01 00:00:00'), 125552498, 2],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 0],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 1],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 2],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 3],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 4],
 [Timestamp('2020-02-01 00:00:00'), 125779698, 5],
 [Timestamp('2020-02-01 00:00:00'), 125864962, 0],
 [Timestamp('2020-02-01 00:00:0

In [8]:
mp_info_dict

{'action': {Timestamp('2020-02-01 00:00:00'): {125119169: {3:       action_type  courier_id       date  expect_time          tracking_id  \
    21123      ASSIGN   125119169 2020-02-01   1580542794  2100074564270097875   
    21124      ASSIGN   125119169 2020-02-01   1580542795  2100074565561812830   
    21125      ASSIGN   125119169 2020-02-01   1580543214  2100074564889904899   
    21126      PICKUP   125119169 2020-02-01   1580543782  2100074565561812830   
    21127      PICKUP   125119169 2020-02-01   1580543784  2100074564270097875   
    21128    DELIVERY   125119169 2020-02-01   1580544001  2100074564270097875   
    21129    DELIVERY   125119169 2020-02-01   1580544372  2100074565561812830   
    21130      PICKUP   125119169 2020-02-01   1580545050  2100074564889904899   
    21131    DELIVERY   125119169 2020-02-01   1580545811  2100074564889904899   
    
           wave_index  
    21123           3  
    21124           3  
    21125           3  
    21126           3

In [9]:
data_info_dict

{'know_lens_list': array([list([5, 6]), list([5, 6]), list([2]), list([7, 8, 9, 10, 11, 12]),
        list([5, 6]), list([2]), list([2]), list([2]), list([5, 6]),
        list([5, 6]), list([2]), list([7, 8, 9, 10, 11, 12]),
        list([6, 7, 8, 9]), list([2]), list([2]), list([2]), list([2]),
        list([2]), list([2]), list([7, 8, 9, 10, 11, 12]), list([2]),
        list([2]), list([2]), list([5, 6]), list([5, 6]), list([2])],
       dtype=object),
 'full_lens': array([ 9,  9,  6, 15,  9,  6,  6,  6,  9,  9,  6, 15, 12,  6,  6,  6,  6,
         6,  6, 15,  6,  6,  6,  9,  9,  6], dtype=int64),
 'impossible_idxs_list': array([{5: {3}, 6: {2}}, {5: {3}, 6: {2}}, {2: {2, 3}},
        {7: {6, 7}, 8: {5, 6}, 9: {4, 5}, 10: {3, 4}, 11: {2, 3}, 12: {2}},
        {5: {3}, 6: {2}}, {2: {1, 3}}, {2: {1, 3}}, {2: {2, 3}},
        {5: {2}, 6: set()}, {5: {1}, 6: set()}, {2: {1, 3}},
        {7: {2, 5, 7}, 8: {1, 6}, 9: {5}, 10: {4}, 11: {3}, 12: {2}},
        {6: {4, 5}, 7: {3, 4}, 8: {2, 3}

In [10]:
len(data_info_dict['full_lens'])

26

In [11]:
mp_info_dict['distance']


{Timestamp('2020-02-01 00:00:00'): {125119169: {3: {2100074564270097875: {2100074564270097875:         courier_id  wave_index          tracking_id source_type  source_lng  \
     213737   125119169           3  2100074564270097875      ASSIGN  121.486737   
     213738   125119169           3  2100074564270097875      ASSIGN  121.486737   
     213746   125119169           3  2100074564270097875      PICKUP  121.485406   
     213747   125119169           3  2100074564270097875    DELIVERY  121.485234   
     213784   125119169           3  2100074564270097875      PICKUP  121.485406   
     213789   125119169           3  2100074564270097875    DELIVERY  121.485234   
     
             source_lat   target_tracking_id target_type  target_lng  target_lat  \
     213737   39.130411  2100074564270097875    DELIVERY  121.485234   39.128801   
     213738   39.130411  2100074564270097875      PICKUP  121.485406   39.129160   
     213746   39.129160  2100074564270097875      ASSIGN  121.48

In [12]:
date = datetime.datetime(2020, 2, 1)
mp_info_dict['distance'][date][125119169][3][2100074564270097875][2100074564270097875]

,courier_id,wave_index,tracking_id,source_type,source_lng,source_lat,target_tracking_id,target_type,target_lng,target_lat,grid_distance,date
213737,125119169,3,2100074564270097875,ASSIGN,121.486737,39.130411,2100074564270097875,DELIVERY,121.485234,39.128801,338.0,2020-02-01
213738,125119169,3,2100074564270097875,ASSIGN,121.486737,39.130411,2100074564270097875,PICKUP,121.485406,39.129160,231.0,2020-02-01
213746,125119169,3,2100074564270097875,PICKUP,121.485406,39.129160,2100074564270097875,ASSIGN,121.486737,39.130411,231.0,2020-02-01
213747,125119169,3,2100074564270097875,DELIVERY,121.485234,39.128801,2100074564270097875,ASSIGN,121.486737,39.130411,338.0,2020-02-01
213784,125119169,3,2100074564270097875,PICKUP,121.485406,39.129160,2100074564270097875,DELIVERY,121.485234,39.128801,61.0,2020-02-01
213789,125119169,3,2100074564270097875,DELIVERY,121.485234,39.128801,2100074564270097875,PICKUP,121.485406,39.129160,41.0,2020-02-01
